<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/FESC_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install transformers datasets evaluate

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [10]:
%%capture
!pip install gdown

In [11]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%capture
!gdown --id 1EZTvcHo2x3cim4uSTkfMHoagipfQtLak
time.sleep(5)

!unzip /content/FESC_CaFE.zip

In [7]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/FESC/FESC_splits/ANRO/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/FESC/FESC_splits/ANRO/test.json",'r') as f:
    test_data = json.load(f)

In [8]:
train_data

{'TIRE_3872_1': {'file_path': 'FESC/16kh_resampled/TIRE_3872_1.WAV',
  'length': 3.4863945578231292,
  'label': '1'},
 'TUVA_4089_3': {'file_path': 'FESC/16kh_resampled/TUVA_4089_3.WAV',
  'length': 3.5664399092970522,
  'label': '3'},
 'TIRE_3623_5': {'file_path': 'FESC/16kh_resampled/TIRE_3623_5.WAV',
  'length': 3.7662585034013607,
  'label': '5'},
 'TIPE_3131_3': {'file_path': 'FESC/16kh_resampled/TIPE_3131_3.WAV',
  'length': 3.3277551020408165,
  'label': '3'},
 'TIRE_3615_1': {'file_path': 'FESC/16kh_resampled/TIRE_3615_1.WAV',
  'length': 8.525532879818595,
  'label': '1'},
 'TUVA_4009_1': {'file_path': 'FESC/16kh_resampled/TUVA_4009_1.WAV',
  'length': 3.4434467120181407,
  'label': '1'},
 'RIGR_2895_4': {'file_path': 'FESC/16kh_resampled/RIGR_2895_4.WAV',
  'length': 3.3902947845804987,
  'label': '4'},
 'JARA_1035_5': {'file_path': 'FESC/16kh_resampled/JARA_1035_5.WAV',
  'length': 2.7300680272108844,
  'label': '5'},
 'MIKO_1611_3': {'file_path': 'FESC/16kh_resampled/MIKO_1

In [9]:
test_data

{'ANRO_256_1': {'file_path': 'FESC/16kh_resampled/ANRO_256_1.WAV',
  'length': 2.125079365079365,
  'label': '1'},
 'ANRO_313_4': {'file_path': 'FESC/16kh_resampled/ANRO_313_4.WAV',
  'length': 1.9469387755102041,
  'label': '4'},
 'ANRO_273_3': {'file_path': 'FESC/16kh_resampled/ANRO_273_3.WAV',
  'length': 2.0040362811791383,
  'label': '3'},
 'ANRO_418_5': {'file_path': 'FESC/16kh_resampled/ANRO_418_5.WAV',
  'length': 2.200544217687075,
  'label': '5'},
 'ANRO_286_1': {'file_path': 'FESC/16kh_resampled/ANRO_286_1.WAV',
  'length': 1.9787755102040816,
  'label': '1'},
 'ANRO_362_5': {'file_path': 'FESC/16kh_resampled/ANRO_362_5.WAV',
  'length': 2.838140589569161,
  'label': '5'},
 'ANRO_78_2': {'file_path': 'FESC/16kh_resampled/ANRO_78_2.WAV',
  'length': 3.394467120181406,
  'label': '2'},
 'ANRO_308_1': {'file_path': 'FESC/16kh_resampled/ANRO_308_1.WAV',
  'length': 6.325260770975056,
  'label': '1'},
 'ANRO_306_1': {'file_path': 'FESC/16kh_resampled/ANRO_306_1.WAV',
  'length': 

In [10]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)

In [11]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [12]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'file_path', 'length', 'label'],
        num_rows: 3229
    })
    test: Dataset({
        features: ['id', 'file_path', 'length', 'label'],
        num_rows: 450
    })
    dev: Dataset({
        features: ['id', 'file_path', 'length', 'label'],
        num_rows: 450
    })
})

In [13]:
def add_base_path(example):
    example['file_path'] = '/content/' + example['file_path']  # Adjust column name if it's not 'wav'
    return example

dataset = dataset.map(add_base_path, batched=False)
dataset['train'][0]

Map:   0%|          | 0/3229 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

{'id': 'TIRE_3872_1',
 'file_path': '/content/FESC/16kh_resampled/TIRE_3872_1.WAV',
 'length': 3.4863945578231292,
 'label': '1'}

In [14]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['file_path'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [15]:
dataset = dataset.map(lambda x: load_audio(x))


Map:   0%|          | 0/3229 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'file_path', 'length', 'label', 'audio_array', 'sample_rate'],
        num_rows: 3229
    })
    test: Dataset({
        features: ['id', 'file_path', 'length', 'label', 'audio_array', 'sample_rate'],
        num_rows: 450
    })
    dev: Dataset({
        features: ['id', 'file_path', 'length', 'label', 'audio_array', 'sample_rate'],
        num_rows: 450
    })
})

In [18]:
dataset['train'][0].keys()

dict_keys(['id', 'file_path', 'length', 'label', 'audio_array', 'sample_rate'])

In [19]:

!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter
!pip install tqdm
from tqdm import tqdm

In [47]:
!pip install transformers datasets
import torch
from torch.nn.utils.rnn import pad_sequence

# Metrics functions
def calculate_unweighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='macro')

def calculate_weighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='weighted')

def calculate_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

# Collate function for DataLoader
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)
    inputs = [torch.tensor(example["input_values"]) for example in batch]
    labels = [int(example["label"]) for example in batch]
    inputs = pad_sequence(inputs, batch_first=True)
    return {"input_values": inputs, "label": torch.tensor(labels)}

    # Example helper function to update paths
def update_FESC_path(path):
    return "/content/" + path  # Adjust as necessary for your file path

In [48]:
%%capture
!git clone https://github.com/aalto-speech/mtkd4ser.git

In [2]:
def update_fesc_label(label, column="emo"):
    if label == "1":
        return "anger"
    elif label == "2":
        return "happiness"
    elif label == "3":
        return "neutral"
    elif label == "4":
        return "sadness"


In [3]:
def to_label(emo, label2id):
    return label2id[emo]

In [4]:
def is_common(emotion):
    if emotion == "neutral" or emotion == "sadness" or emotion == "happiness" or emotion == "anger":
        return 1
    else:
        return 0

In [5]:
import random # import the random module
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from transformers import Wav2Vec2FeatureExtractor
from torch.utils.data import DataLoader
import os
from sklearn.metrics import recall_score, accuracy_score
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
# Finnish Spoken Emotion
def fesc(session):
    SESSION_FOLDER_MAP = {
        1: "TIRE", 2: "TIPE", 3: "JARA", 4: "MIKO", 5: "ANRO",
        6: "RIGR", 7: "TUVA", 8: "PEKO", 9: "JAKA"
    }
    FOLDER_NAME = SESSION_FOLDER_MAP.get(session, "ERROR")

    train_json_file = f"/content/FESC/FESC_splits/{FOLDER_NAME}/train.json"
    test_json_file = f"/content/FESC/FESC_splits/{FOLDER_NAME}/test.json"
    dev_json_file = f"/content/FESC/FESC_splits/{FOLDER_NAME}/dev.json"

    with open(train_json_file, "r") as f:
        train_data = json.load(f)
    with open(test_json_file, "r") as f:
        test_data = json.load(f)
    with open(dev_json_file, "r") as f:
        dev_data = json.load(f)


    train_df = pd.DataFrame.from_dict(train_data, orient="index").reset_index()
    test_df = pd.DataFrame.from_dict(test_data, orient="index").reset_index()
    dev_df = pd.DataFrame.from_dict(dev_data, orient="index").reset_index()

    train_df["audio"] = train_df["file_path"].apply(update_FESC_path)
    test_df["audio"] = test_df["file_path"].apply(update_FESC_path)
    dev_df["audio"] = dev_df["file_path"].apply(update_FESC_path)

    train_df = train_df.rename(columns={'index': 'file_id', 'label': 'emo'})
    test_df = test_df.rename(columns={'index': 'file_id', 'label': 'emo'})
    dev_df = dev_df.rename(columns={'index': 'file_id', 'label': 'emo'})

    train_df = train_df.loc[train_df['emo'] != '5'].reset_index(drop=True)
    test_df = test_df.loc[test_df['emo'] != '5'].reset_index(drop=True)
    dev_df = dev_df.loc[dev_df['emo'] != '5'].reset_index(drop=True)

    train_df["emo"] = train_df["emo"].apply(update_fesc_label)
    test_df["emo"] = test_df["emo"].apply(update_fesc_label)
    dev_df["emo"] = dev_df["emo"].apply(update_fesc_label)

    labels = ['anger', 'happiness', 'neutral', 'sadness'] # ["1", "2", "3", "4"]
    label2id, id2label = dict(), dict()

    for i, label in enumerate(labels):
        label2id[label] = str(i)
        id2label[str(i)] = label

    train_df["label"] = train_df["emo"].apply(to_label, args=(label2id,))
    test_df["label"] = test_df["emo"].apply(to_label, args=(label2id,))
    dev_df["label"] = dev_df["emo"].apply(to_label, args=(label2id,))

    train_audio_data = Dataset.from_pandas(train_df[['audio', 'label']])
    train_audio_data = train_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    test_audio_data = Dataset.from_pandas(test_df[['audio', 'label']])
    test_audio_data = test_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    dev_audio_data = Dataset.from_pandas(dev_df[['audio', 'label']])
    dev_audio_data = dev_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    ds = DatasetDict({
        'train' : train_audio_data,
        'test' : test_audio_data,
        'dev' : dev_audio_data
    })

    return label2id, id2label, ds


In [6]:
def preprocess_function(examples, feature_extractor):
    max_duration = 20.0
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=int(feature_extractor.sampling_rate * max_duration), padding=True, truncation=True)
    return inputs


In [54]:
def model_ft(label2id, id2label, num_classes, device="cpu"):
    model_name = "facebook/wav2vec2-base"
    model = AutoModelForAudioClassification.from_pretrained(model_name, num_labels=num_classes, label2id=label2id, id2label=id2label)
    model.to(device)
    return model

def pipeline_ft(model, train_loader, valid_loader, test_loader, hyperparam, device):
    SESSION = hyperparam['SESSION']
    LEARNING_RATE = hyperparam['LEARNING_RATE']
    TRAINING = hyperparam['TRAINING']
    N_EPOCHS = hyperparam['N_EPOCHS']
    LINGUALITY = hyperparam['LINGUALITY']
    LANGUAGE = hyperparam['LANGUAGE']

    file_path = f"/content/FT_{LINGUALITY}_{LANGUAGE}_S{SESSION}.pth"
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    # Load checkpoint if exists
    epoch = 1
    if os.path.exists(file_path):
        checkpoint = torch.load(file_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        total_train_loss = checkpoint.get('training loss', 0)
        total_valid_loss = checkpoint.get('validation loss', 0)
        print("Model checkpoint has been loaded")

    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir=f"/content/tensorboards/FT_{LINGUALITY}_{LANGUAGE}_S{SESSION}")

    # Evaluation-only mode
    if TRAINING == 0:
        test_unweighted_recall, test_weighted_recall, test_accuracy = validation(
            model,
            tqdm(test_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Test", leave=False),
            loss_fn,
            device
        )
        print(f"Epoch {epoch}/{N_EPOCHS}, Test Recall (unweighted): {test_unweighted_recall}, Recall (weighted): {test_weighted_recall}, Accuracy: {test_accuracy}\n")

    else:
        # Training and validation mode
        for epoch in range(epoch, N_EPOCHS+1):
            # Train step - adjust to returned values
            train_unweighted_recall, train_weighted_recall, train_accuracy = train(
                model,
                tqdm(train_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Training", leave=False),
                optimizer,
                loss_fn,
                device
            )
            print(f"Epoch {epoch}/{N_EPOCHS}, Training Recall (unweighted): {train_unweighted_recall}, Recall (weighted): {train_weighted_recall}, Accuracy: {train_accuracy}")

            # Log training metrics
            writer.add_scalar("Accuracy/Train", train_accuracy, epoch)
            writer.add_scalar("Recall_Unweighted/Train", train_unweighted_recall, epoch)
            writer.add_scalar("Recall_Weighted/Train", train_weighted_recall, epoch)

            # Validation step - adjust to returned values
            valid_unweighted_recall, valid_weighted_recall, valid_accuracy = validation(
                model,
                tqdm(valid_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Validation", leave=False),
                loss_fn,
                device
            )
            print(f"Epoch {epoch}/{N_EPOCHS}, Validation Recall (unweighted): {valid_unweighted_recall}, Recall (weighted): {valid_weighted_recall}, Accuracy: {valid_accuracy}\n")

            # Log validation metrics
            writer.add_scalar("Accuracy/Validation", valid_accuracy, epoch)
            writer.add_scalar("Recall_Unweighted/Validation", valid_unweighted_recall, epoch)
            writer.add_scalar("Recall_Weighted/Validation", valid_weighted_recall, epoch)

            writer.flush()

            # Save model checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'training loss': train_unweighted_recall,  # Adjust as needed
                'validation loss': valid_unweighted_recall  # Adjust as needed
            }, file_path)

            print(f"Model has been saved after epoch: {epoch}\n")

        # Close TensorBoard writer
        writer.close()

def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_train_loss, correct_predictions, total_predictions = 0, 0, 0
    all_actual_labels, all_predictions = [], []

    for batch in train_loader:
        inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs).logits
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
        all_actual_labels.extend(labels.tolist())
        all_predictions.extend(predicted.tolist())

    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    return unweighted_recall, correct_predictions / total_predictions, total_train_loss / len(train_loader)
# Define the validation function
def validation(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in data_loader:
            inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
            outputs = model(inputs).logits
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            predictions = torch.argmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    unweighted_recall = recall_score(all_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_labels, all_predictions, average='weighted')
    accuracy = sum(np.array(all_labels) == np.array(all_predictions)) / len(all_labels)

    return unweighted_recall, weighted_recall, accuracy

In [55]:
# Define the Args class with instance variables
class Args:
    def __init__(self):
        self.N_EPOCHS = 10
        self.LEARNING_RATE = 1e-5
        self.BATCH_SIZE = 8
        self.SESSION = 1
        self.TRAINING = 1
        self.PARADIGM = "FT"
        self.LANGUAGE = "FR"
        self.LINGUALITY = "Monolingual"

In [56]:
def main(args_dict):
    random_seed = 42
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(random_seed)

    # Access values using args_dict
    session = args_dict["SESSION"]
    batch_size = args_dict["BATCH_SIZE"]
    learning_rate = args_dict["LEARNING_RATE"]

    # Load the CaFE dataset
    label2id, id2label, ds = fesc(session)

    # Initialize feature extractor
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")

    # Preprocess dataset
    encoded_audio = ds.map(lambda example: preprocess_function(example, feature_extractor), remove_columns="audio", batched=True)

    # Initialize DataLoaders
    train_loader = DataLoader(encoded_audio["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(encoded_audio["test"], batch_size=batch_size, collate_fn=collate_fn)
    valid_loader = DataLoader(encoded_audio["dev"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # Model and optimizer
    model = model_ft(label2id, id2label, len(label2id), device="cuda" if torch.cuda.is_available() else "cpu")
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    # Run pipeline
    pipeline_ft(model, train_loader, valid_loader, test_loader, args_dict, device="cuda" if torch.cuda.is_available() else "cpu")


In [1]:
# Create an instance of Args and pass it as a dictionary to main
args = Args()
main(args.__dict__)

NameError: name 'Args' is not defined